In [87]:
import pandas as pd

In [88]:
df = pd.read_csv('../data/raw/2-ExploracionDataframes.csv', sep=';')
df

,CUT Comuna Origen,Comuna Origen,CUT Provincia Origen,Provincia Origen,CUT Region Origen,Region Origen,CUT Comuna Destino,Comuna Destino,Destino Turistico,CUT Provincia Destino,Provincia Destino,CUT Region Destino,Region Destino,Anio,CUT Mes,Mes,Viajes Ocasionales,Temporada,PIB Region Origen,PIB Region Destino
0,1101,Iquique,11,Iquique,1,Tarapacá,1402,Camiña,Camiña,14,Tamarugal,1,Tarapacá,2019,1,enero,109.0,Alta,1061.580205,1061.580205
1,1101,Iquique,11,Iquique,1,Tarapacá,1403,Colchane,Colchane - P.N. Volcán Isluga,14,Tamarugal,1,Tarapacá,2019,1,enero,298.0,Alta,1061.580205,1061.580205
2,1101,Iquique,11,Iquique,1,Tarapacá,1404,Huara,Resto región Tarapacá,14,Tamarugal,1,Tarapacá,2019,1,enero,2594.0,Alta,1061.580205,1061.580205
3,1101,Iquique,11,Iquique,1,Tarapacá,1405,Pica,Pica,14,Tamarugal,1,Tarapacá,2019,1,enero,4138.0,Alta,1061.580205,1061.580205
4,1101,Iquique,11,Iquique,1,Tarapacá,2101,Antofagasta,Antofagasta,21,Antofagasta,2,Antofagasta,2019,1,enero,2071.0,Alta,1061.580205,3724.530431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6489667,16305,San Nicolás,163,Punilla,16,Ñuble,16203,Coelemu,Valle del Itata,162,Itata,16,Ñuble,2024,12,diciembre,NaN,Baja,NaN,NaN
6489668,16305,San Nicolás,163,Punilla,16,Ñuble,16207,Treguaco,Resto región Ñuble,162,Itata,16,Ñuble,2024,12,diciembre,NaN,Baja,NaN,NaN
6489669,16305,San Nicolás,163,Punilla,16,Ñuble,16302,Coihueco,Resto región Ñuble,163,Punilla,16,Ñuble,2024,12,diciembre,NaN,Baja,NaN,NaN
6489670,16305,San Nicolás,163,Punilla,16,Ñuble,16303,Ñiquén,Resto región Ñuble,163,Punilla,16,Ñuble,2024,12,diciembre,NaN,Baja,NaN,NaN


In [ ]:
# Agrupar los viajes por 'Region Destino' y 'Mes'
viajes_totales_temporada = df.groupby(['Region Destino', 'Provincia Destino','Comuna Destino', 'Anio','CUT Mes','Mes', 'Temporada'])['Viajes Ocasionales'].sum().reset_index()

# Lista de niveles de agregación para los totales
niveles_agrupacion = [
    (['Region Destino', 'Anio', 'Temporada'], 'Total Temporada Region'),
    (['Provincia Destino', 'Anio', 'Temporada'], 'Total Temporada Provincia'),
    (['Comuna Destino', 'Anio', 'Temporada'], 'Total Temporada Comuna')
]

# Iterar sobre los niveles de agrupación y añadir las columnas de totales
for nivel, nombre_columna in niveles_agrupacion:
    # Crear un dataframe con el total de viajes por el nivel de agregación
    total_viajes = viajes_totales_temporada.groupby(nivel)['Viajes Ocasionales'].sum().reset_index()
    total_viajes.rename(columns={'Viajes Ocasionales': nombre_columna}, inplace=True)
    
    # Unir este total al dataframe original
    viajes_totales_temporada = viajes_totales_temporada.merge(total_viajes, on=nivel, how='left')

viajes_totales_temporada.head()

,Region Destino,Provincia Destino,Comuna Destino,Anio,CUT Mes,Mes,Temporada,Viajes Ocasionales,Total Temporada Region,Total Temporada Provincia,Total Temporada Comuna
0,Antofagasta,Antofagasta,Antofagasta,2019,1,enero,Alta,46368.0,367464.0,223712.0,138046.0
1,Antofagasta,Antofagasta,Antofagasta,2019,2,febrero,Alta,36755.0,367464.0,223712.0,138046.0
2,Antofagasta,Antofagasta,Antofagasta,2019,3,marzo,Baja,15596.0,469839.0,271939.0,180410.0
3,Antofagasta,Antofagasta,Antofagasta,2019,4,abril,Baja,17566.0,469839.0,271939.0,180410.0
4,Antofagasta,Antofagasta,Antofagasta,2019,5,mayo,Baja,31094.0,469839.0,271939.0,180410.0


: 

In [ ]:
# Eliminar la columna 'Viajes Ocasionales' del dataframe 'viajes_por_mes_region' antes del merge
viajes_totales_temporada = viajes_totales_temporada.drop(columns=['Viajes Ocasionales'])


# Realizar el merge del dataframe 'viajes_por_mes_region' al dataframe original 'df'
df = df.merge(
    viajes_totales_temporada, 
    on=['Region Destino', 'Provincia Destino', 'Comuna Destino', 'Anio', 'CUT Mes', 'Mes', 'Temporada'], 
    how='left'
)

df

In [ ]:
# Filtrar el dataframe para eliminar filas que cumplen con las condiciones
df_filtrado = df[~((df['Anio'] == 2024) & (df['CUT Mes'] > 6))]
df = df_filtrado
df

,CUT Comuna Origen,Comuna Origen,CUT Provincia Origen,Provincia Origen,CUT Region Origen,Region Origen,CUT Comuna Destino,Comuna Destino,Destino Turistico,CUT Provincia Destino,...,Anio,CUT Mes,Mes,Viajes Ocasionales,Temporada,PIB Region Origen,PIB Region Destino,Total Temporada Region,Total Temporada Provincia,Total Temporada Comuna
0,1101,Iquique,11,Iquique,1,Tarapacá,1402,Camiña,Camiña,14,...,2019,1,enero,109.0,Alta,1061.580205,1061.580205,272938.0,78851.0,1298.0
1,1101,Iquique,11,Iquique,1,Tarapacá,1403,Colchane,Colchane - P.N. Volcán Isluga,14,...,2019,1,enero,298.0,Alta,1061.580205,1061.580205,272938.0,78851.0,6055.0
2,1101,Iquique,11,Iquique,1,Tarapacá,1404,Huara,Resto región Tarapacá,14,...,2019,1,enero,2594.0,Alta,1061.580205,1061.580205,272938.0,78851.0,23124.0
3,1101,Iquique,11,Iquique,1,Tarapacá,1405,Pica,Pica,14,...,2019,1,enero,4138.0,Alta,1061.580205,1061.580205,272938.0,78851.0,22500.0
4,1101,Iquique,11,Iquique,1,Tarapacá,2101,Antofagasta,Antofagasta,21,...,2019,1,enero,2071.0,Alta,1061.580205,3724.530431,367464.0,223712.0,138046.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6489400,16305,San Nicolás,163,Punilla,16,Ñuble,16203,Coelemu,Valle del Itata,162,...,2023,12,diciembre,17.0,Baja,833.567298,833.567298,934460.0,169183.0,25342.0
6489401,16305,San Nicolás,163,Punilla,16,Ñuble,16207,Treguaco,Resto región Ñuble,162,...,2023,12,diciembre,44.0,Baja,833.567298,833.567298,934460.0,169183.0,15079.0
6489402,16305,San Nicolás,163,Punilla,16,Ñuble,16302,Coihueco,Resto región Ñuble,163,...,2023,12,diciembre,108.0,Baja,833.567298,833.567298,934460.0,192250.0,26315.0
6489403,16305,San Nicolás,163,Punilla,16,Ñuble,16303,Ñiquén,Resto región Ñuble,163,...,2023,12,diciembre,47.0,Baja,833.567298,833.567298,934460.0,192250.0,40543.0


In [ ]:
# Supongamos que ya tienes tu dataframe llamado 'df'
# Filtrar filas que contienen al menos un NaN
df_nan = df[df.isna().any(axis=1)]

# Mostrar el dataframe filtrado
df_nan

,CUT Comuna Origen,Comuna Origen,CUT Provincia Origen,Provincia Origen,CUT Region Origen,Region Origen,CUT Comuna Destino,Comuna Destino,Destino Turistico,CUT Provincia Destino,...,Anio,CUT Mes,Mes,Viajes Ocasionales,Temporada,PIB Region Origen,PIB Region Destino,Total Temporada Region,Total Temporada Provincia,Total Temporada Comuna
51,1101,Iquique,11,Iquique,1,Tarapacá,5404,Petorca,Resto región Valparaíso,54,...,2019,1,enero,NaN,Alta,1061.580205,3408.549930,3127724.0,289678.0,6948.0
65,1101,Iquique,11,Iquique,1,Tarapacá,5702,Catemu,Resto región Valparaíso,57,...,2019,1,enero,NaN,Alta,1061.580205,3408.549930,3127724.0,106727.0,5538.0
78,1101,Iquique,11,Iquique,1,Tarapacá,6105,Doñihue,Resto región Libertador General Bernardo O'Hig...,61,...,2019,1,enero,NaN,Alta,1061.580205,2218.904833,1084766.0,498804.0,9183.0
84,1101,Iquique,11,Iquique,1,Tarapacá,6111,Olivar,Resto región Libertador General Bernardo O'Hig...,61,...,2019,1,enero,NaN,Alta,1061.580205,2218.904833,1084766.0,498804.0,7712.0
94,1101,Iquique,11,Iquique,1,Tarapacá,6204,Marchihue,Valle de Colchagua,62,...,2019,1,enero,NaN,Alta,1061.580205,2218.904833,1084766.0,371719.0,18422.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6489391,16305,San Nicolás,163,Punilla,16,Ñuble,14201,La Unión,Cuenca del Lago Ranco,142,...,2023,12,diciembre,NaN,Baja,833.567298,716.073318,865949.0,204283.0,56599.0
6489392,16305,San Nicolás,163,Punilla,16,Ñuble,14202,Futrono,Cuenca del Lago Ranco,142,...,2023,12,diciembre,NaN,Baja,833.567298,716.073318,865949.0,204283.0,51717.0
6489393,16305,San Nicolás,163,Punilla,16,Ñuble,14203,Lago Ranco,Cuenca del Lago Ranco,142,...,2023,12,diciembre,NaN,Baja,833.567298,716.073318,865949.0,204283.0,48476.0
6489394,16305,San Nicolás,163,Punilla,16,Ñuble,14204,Río Bueno,Cuenca del Lago Ranco,142,...,2023,12,diciembre,NaN,Baja,833.567298,716.073318,865949.0,204283.0,47491.0


In [ ]:
# Verificar la cantidad de NaN por columna
nan_counts = df.isna().sum()
print(nan_counts)

CUT Comuna Origen                  0
Comuna Origen                      0
CUT Provincia Origen               0
Provincia Origen                   0
CUT Region Origen                  0
Region Origen                      0
CUT Comuna Destino                 0
Comuna Destino                     0
Destino Turistico                  0
CUT Provincia Destino              0
Provincia Destino                  0
CUT Region Destino                 0
Region Destino                     0
Anio                               0
CUT Mes                            0
Mes                                0
Viajes Ocasionales           2556277
Temporada                          0
PIB Region Origen                  0
PIB Region Destino                 0
Total Temporada Region             0
Total Temporada Provincia          0
Total Temporada Comuna             0
dtype: int64


In [ ]:
for temporada in df['Temporada'].unique():
    for destino in df['Destino Turístico'].unique():
        mask = (df['temporada'] == temporada) & (df['Destino Turístico'] == destino)
        df.loc[mask, 'viajes'] = df.loc[mask, 'viajes'].interpolate(method='linear')

df

In [85]:
df.loc[df['Temporada'] == 'Alta', 'Viajes Ocasionales'] = df.loc[df['Temporada'] == 'Alta', 'Viajes Ocasionales'].interpolate(method='linear')

# Filtrar por temporada baja y aplicar interpolación lineal solo dentro de esta temporada
df.loc[df['Temporada'] == 'Baja', 'Viajes Ocasionales'] = df.loc[df['Temporada'] == 'Baja', 'Viajes Ocasionales'].interpolate(method='linear')

In [86]:
# Imprimir la fila en el índice 5 (por ejemplo)
fila_especifica = df.iloc[51]
print(fila_especifica)

CUT Comuna Origen                               1101
Comuna Origen                                Iquique
CUT Provincia Origen                              11
Provincia Origen                             Iquique
CUT Region Origen                                  1
Region Origen                               Tarapacá
CUT Comuna Destino                              5404
Comuna Destino                               Petorca
Destino Turistico            Resto región Valparaíso
CUT Provincia Destino                             54
Provincia Destino                            Petorca
CUT Region Destino                                 5
Region Destino                            Valparaíso
Anio                                            2019
CUT Mes                                            1
Mes                                            enero
Viajes Ocasionales                              15.0
Temporada                                       Alta
PIB Region Origen                        1061.

In [74]:
df

,CUT Comuna Origen,Comuna Origen,CUT Provincia Origen,Provincia Origen,CUT Region Origen,Region Origen,CUT Comuna Destino,Comuna Destino,Destino Turistico,CUT Provincia Destino,...,Anio,CUT Mes,Mes,Viajes Ocasionales,Temporada,PIB Region Origen,PIB Region Destino,Total Temporada Region,Total Temporada Provincia,Total Temporada Comuna
0,1101,Iquique,11,Iquique,1,Tarapacá,1402,Camiña,Camiña,14,...,2019,1,enero,109.0,Alta,1061.580205,1061.580205,272938.0,78851.0,1298.0
1,1101,Iquique,11,Iquique,1,Tarapacá,1403,Colchane,Colchane - P.N. Volcán Isluga,14,...,2019,1,enero,298.0,Alta,1061.580205,1061.580205,272938.0,78851.0,6055.0
2,1101,Iquique,11,Iquique,1,Tarapacá,1404,Huara,Resto región Tarapacá,14,...,2019,1,enero,2594.0,Alta,1061.580205,1061.580205,272938.0,78851.0,23124.0
3,1101,Iquique,11,Iquique,1,Tarapacá,1405,Pica,Pica,14,...,2019,1,enero,4138.0,Alta,1061.580205,1061.580205,272938.0,78851.0,22500.0
4,1101,Iquique,11,Iquique,1,Tarapacá,2101,Antofagasta,Antofagasta,21,...,2019,1,enero,2071.0,Alta,1061.580205,3724.530431,367464.0,223712.0,138046.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6489400,16305,San Nicolás,163,Punilla,16,Ñuble,16203,Coelemu,Valle del Itata,162,...,2023,12,diciembre,17.0,Baja,833.567298,833.567298,934460.0,169183.0,25342.0
6489401,16305,San Nicolás,163,Punilla,16,Ñuble,16207,Treguaco,Resto región Ñuble,162,...,2023,12,diciembre,44.0,Baja,833.567298,833.567298,934460.0,169183.0,15079.0
6489402,16305,San Nicolás,163,Punilla,16,Ñuble,16302,Coihueco,Resto región Ñuble,163,...,2023,12,diciembre,108.0,Baja,833.567298,833.567298,934460.0,192250.0,26315.0
6489403,16305,San Nicolás,163,Punilla,16,Ñuble,16303,Ñiquén,Resto región Ñuble,163,...,2023,12,diciembre,47.0,Baja,833.567298,833.567298,934460.0,192250.0,40543.0


In [75]:
df.to_csv('../data/processed/df_listo_para_eda.csv', sep=';', index=False)